# Práctica 4: Procesamieto de audio

### **Participantes:**
- Gerardo León Quintana
- Susana Suárez Mendoza

## Ejercicio 1:

Construir un identificador de notas musicales. Es decir; en su versión más sencilla (y suficiente) la entrada es un sonido con una sola nota musical y debe identificar cuál es. Por simplicidad elija un único instrumento para la identificación.

## Ejercicio 2:

Construir una pequeña aplicación que permita operar con diferentes filtros (con un selector) y trabajar con varios umbrales. (uno para los filtros pasa-bajo y pasa-alto y dos para los filtros pasa-banda y rechaza-banda). Demuestre su funcionalidad con señales ruidosas. Muestre en cada filtrado la señal original y filtrada en el dominio temporal y en el dominio de la frecuencia.

# gerardo branch

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from scipy.io import wavfile
import tkinter as tk
from tkinter import filedialog
import vlc
from pydub import AudioSegment
import tempfile
import os
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import warnings
warnings.filterwarnings('ignore')


In [6]:
# Función para aplicar filtros
def apply_filter(data, filter_type, cutoff, fs, order=5, band=None):
    nyquist = 0.5 * fs
    if filter_type == 'low':
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
    elif filter_type == 'high':
        normal_cutoff = cutoff / nyquist
        b, a = butter(order, normal_cutoff, btype='high', analog=False)
    elif filter_type == 'band':
        low, high = band
        low /= nyquist
        high /= nyquist
        b, a = butter(order, [low, high], btype='band', analog=False)
    elif filter_type == 'notch':
        low, high = band
        low /= nyquist
        high /= nyquist
        b, a = butter(order, [low, high], btype='bandstop', analog=False)
    y = lfilter(b, a, data)
    return y

In [7]:
class AudioPlayer:
    def __init__(self, master):
        self.master = master
        self.master.title("Reproductor de Audio")
        self.master.geometry("800x600")

        # Inicializar el reproductor VLC
        self.player = None
        self.is_playing = False

        # Botón para elegir archivo
        self.select_button = tk.Button(self.master, text="Seleccionar Archivo", command=self.select_file)
        self.select_button.pack(pady=10)

        # Opciones de filtros
        self.filter_type = tk.StringVar(value="none")
        self.cutoff = tk.DoubleVar(value=1000.0)
        self.low_cutoff = tk.DoubleVar(value=500.0)
        self.high_cutoff = tk.DoubleVar(value=2000.0)

        tk.Label(self.master, text="Seleccionar Filtro:").pack()
        filters = [("Ninguno", "none"), ("Pasa-Bajo", "low"), ("Pasa-Alto", "high"), ("Pasa-Banda", "band"), ("Rechaza-Banda", "notch")]
        for text, mode in filters:
            tk.Radiobutton(self.master, text=text, variable=self.filter_type, value=mode, command=self.apply_selected_filter).pack(anchor=tk.W)

        # Control de la frecuencia de corte
        tk.Label(self.master, text="Frecuencia de Corte (Hz):").pack()
        self.cutoff_scale = tk.Scale(self.master, variable=self.cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.cutoff_scale.pack()

        # Control de la frecuencia baja
        tk.Label(self.master, text="Frecuencia Baja (Hz):").pack()
        self.low_cutoff_scale = tk.Scale(self.master, variable=self.low_cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.low_cutoff_scale.pack()

        # Control de la frecuencia alta
        tk.Label(self.master, text="Frecuencia Alta (Hz):").pack()
        self.high_cutoff_scale = tk.Scale(self.master, variable=self.high_cutoff, from_=20, to_=20000, orient=tk.HORIZONTAL, command=self.apply_selected_filter)
        self.high_cutoff_scale.pack()

        # Botón de reproducir/pausar
        self.play_button = tk.Button(self.master, text="Reproducir", command=self.toggle_play, state=tk.DISABLED)
        self.play_button.pack(pady=20)

        # Botón para guardar archivo filtrado
        self.save_button = tk.Button(self.master, text="Guardar Archivo Filtrado", command=self.save_file, state=tk.DISABLED)
        self.save_button.pack(pady=10)

        self.filtered_file = None
        self.original_file = None 

        # Control de volumen
        self.volume_var = tk.DoubleVar(value=100)
        self.volume_scale = tk.Scale(self.master, label="Volumen", variable=self.volume_var, from_=0, to_=100, orient=tk.HORIZONTAL, command=self.set_volume)
        self.volume_scale.pack(fill=tk.X, padx=10, pady=10)

        # Visualización de la señal de audio
        self.fig = Figure(figsize=(6, 2))
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.master)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=1)

    def select_file(self):
        # Abrir un cuadro de diálogo para seleccionar un archivo de audio
        file_path = filedialog.askopenfilename(title="Seleccionar Archivo de Audio", filetypes=[("Audio Files", "*.mp3;*.wav;*.ogg;*.flac")])
        if file_path:
            self.original_file = file_path
            self.apply_selected_filter()

    def apply_selected_filter(self, *args):
        # Aplicar el filtro seleccionado al audio
        if self.original_file:
            if self.player is not None and self.is_playing:
                self.player.pause()
                self.is_playing = False
                self.play_button.config(text="Reproducir")

            
            audio = AudioSegment.from_file(self.original_file)
            samples = np.array(audio.get_array_of_samples())
            fs = audio.frame_rate

            filter_type = self.filter_type.get()
            cutoff = self.cutoff.get()
            low_cutoff = self.low_cutoff.get()
            high_cutoff = self.high_cutoff.get()
            if filter_type != "none":
                if filter_type in ["band", "notch"]:
                    filtered_samples = apply_filter(samples, filter_type, None, fs, band=(low_cutoff, high_cutoff))
                else:
                    filtered_samples = apply_filter(samples, filter_type, cutoff, fs)
                self.update_waveform(filtered_samples, fs)
                filtered_audio = audio._spawn(filtered_samples.astype(np.int16).tobytes())
            else:
                self.update_waveform(samples, fs)
                filtered_audio = audio

            # Guardar el audio filtrado en un archivo temporal para reproducirlo en VLC
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_file:
                filtered_audio.export(temp_file.name, format="wav")
                self.filtered_file = temp_file.name

            self.player = vlc.MediaPlayer(self.filtered_file)
            self.play_button.config(state=tk.NORMAL)
            self.save_button.config(state=tk.NORMAL)

    def toggle_play(self):
        if self.player is None:
            return

        if self.is_playing:
            self.player.pause()
            self.play_button.config(text="Continuar")
        else:
            if self.player.is_playing():
                self.player.stop()
            self.player.play()
            self.play_button.config(text="Pausar")
        self.is_playing = not self.is_playing

    def set_volume(self, volume):
        if self.player:
            self.player.audio_set_volume(int(volume))

    def save_file(self):
        if self.filtered_file:
            save_path = filedialog.asksaveasfilename(title="filtered_audio", defaultextension=".wav",
                                                     filetypes=[("WAV Files", "*.wav")])
            if save_path:
                try:
                    with open(self.filtered_file, 'rb') as temp_file:
                        with open(save_path, 'wb') as out_file:
                            out_file.write(temp_file.read())
                    messagebox.showinfo("Guardar Archivo", "Archivo guardado exitosamente.")
                except Exception as e:
                    messagebox.showerror("Error", f"No se pudo guardar el archivo: {e}")

    def update_waveform(self, samples, fs):
        self.ax.clear()
        self.ax.plot(np.linspace(0, len(samples) / fs, num=len(samples)), samples)
        self.ax.set_xlabel("Tiempo [s]")
        self.ax.set_ylabel("Amplitud")
        self.canvas.draw()

    def __del__(self):
        if self.filtered_file:
            os.remove(self.filtered_file) 

if __name__ == "__main__":
    root = tk.Tk()
    app = AudioPlayer(root)
    root.mainloop()


Exception ignored in: <function AudioPlayer.__del__ at 0x000001D05A7B36D0>
Traceback (most recent call last):
  File "C:\Users\gerar\AppData\Local\Temp\ipykernel_26068\3446859762.py", line 147, in __del__
PermissionError: [WinError 32] El proceso no tiene acceso al archivo porque está siendo utilizado por otro proceso: 'C:\\Users\\gerar\\AppData\\Local\\Temp\\tmpu9oxcky2.wav'
